In [1]:
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [2]:
# Reading with pandas didn't work, so doing this instead:
#
# IMPORTANT: the delimiter for the county csv data is ;
def read_csv(path, delim):
    with open(path) as f:
        reader = csv.reader(f, delimiter=delim)
        rows = list(reader)
        header= rows[0]
        data = rows[1:]
        return header, data

In [3]:
path_to_csv = "./usa-2016-presidential-election-by-county.csv" # replace with your path
header, data = read_csv(path_to_csv, ';') 

In [4]:
# Compute state average for each feature
all_data = pd.DataFrame(data, columns=header)
states = list(set(all_data["State"]))
state_average = {s:{f:None for f in header} for s in states}
for feature in header[2:]:
    for state in states:
        missing = all_data[(all_data["State"] == state) & (all_data[feature] == '')]
        if not missing.empty:
            try:
                avg = all_data[(all_data["State"] == state) & (all_data[feature] != '')][feature].astype(np.float32)
                # Do average over all states if it is empty
                if avg.empty:
                    avg = all_data[(all_data[feature] != '')][feature].astype(np.float32)
                state_average[state][feature] = avg.mean()
                #print(state_average[state][feature], print(avg))
                
            except:
                print("Feature {} is not numeric".format(feature))


Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Temp Bins is not numeric
Feature Te

In [5]:
# Use better function later maybe
def fill_in_row(row):
    state = row['State']
    for key in row.index:
        if row[key] == '':
            row[key] = state_average[state][key]
    return row
            
all_data.apply(fill_in_row, axis = 1)

,State,ST,Fips,County,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),...,total08,total12,other08,other12,Other12 Frac,Other08 Frac,Rep12 Frac2,Rep08 Frac2,Dem12 Frac2,Dem08 Frac2
0,Georgia,GA,13231,"Pike County, Georgia",8,8675,1575,1356,6547,6668,...,8203,8109,81,85,0.010482180293501,0.009874436181884001,0.8310069790628111,0.806082245752277,0.16899302093718802,0.193917754247722
1,Utah,UT,49051,"Wasatch County, Utah",38,12120,2892,2191,5430,7220,...,8531,9589,209,178,0.0185629366983,0.024498886414253,0.767187333970885,0.6524873828406631,0.23281266602911402,0.347512617159336
2,Washington,WA,53021,"Franklin County, Washington",15,20584,7361,8398,12037,13748,...,19696,22643,298,497,0.021949388331934,0.015129975629569001,0.6207893073241211,0.6205278894731411,0.37921069267587804,0.37947211052685803
3,Nebraska,NE,31119,"Madison County, Nebraska",22,13956,4142,3485,9655,10062,...,14045,13885,248,338,0.024342815988476003,0.017657529369882,0.742747471764966,0.69978980937885,0.25725252823503303,0.300210190621149
4,Florida,FL,12045,"Gulf County, Florida",9,7279,2149,2014,4980,4995,...,7205,7108,76,99,0.013927968486212002,0.010548230395558,0.7126551576544441,0.698555197082339,0.287344842345555,0.30144480291766
5,Georgia,GA,13267,"Tattnall County, Georgia",8,6879,1932,1897,4730,4706,...,6714,6667,52,64,0.009599520023998,0.007745010425975001,0.7127063456004841,0.7099969978985291,0.28729365439951504,0.290003002101471
6,Wisconsin,WI,55083,"Oconto County, Wisconsin",28,19924,9927,8865,8755,10741,...,18968,19859,286,253,0.012739815700689001,0.015078026149304001,0.547842497194736,0.468632908682153,0.452157502805263,0.5313670913178461
7,Alabama,AL,01059,"Franklin County, Alabama",26,11955,3469,3171,8048,7567,...,11693,10881,176,143,0.013142174432497001,0.015051740357478001,0.704693611473272,0.698793088477902,0.295306388526727,0.301206911522097
8,Washington,WA,53015,"Cowlitz County, Washington",30,45485,24597,22726,19554,20746,...,45229,44620,1078,1148,0.025728372926938,0.023834265626036,0.47722672064777305,0.44288917578310805,0.5227732793522261,0.5571108242168921
9,Missouri,MO,29061,"Daviess County, Missouri",7,3689,1400,1125,2263,2290,...,3786,3521,123,106,0.030105083783016004,0.032488114104595005,0.6705710102489011,0.617799617799617,0.329428989751098,0.38220038220038205


In [6]:
numeric_features = []
for feature in header:
    try:
        subtable = all_data[feature].astype(np.float32)
        numeric_features.append(feature)
    except:
        pass
numeric_features.remove("Lat Bins")
fips = all_data["Fips"].values
df = all_data[numeric_features].astype(np.float32)


# Normalize data
cols = df.columns
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized_df = pd.DataFrame(x_scaled)
normalized_df.columns = cols
normalized_df['Fips'] = pd.Series(fips)
normalized_df.head() # Normalized data

,Fips,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),Republicans 2016,Democrats 2016,Green 2016,...,total08,total12,other08,other12,Other12 Frac,Other08 Frac,Rep12 Frac2,Rep08 Frac2,Dem12 Frac2,Dem08 Frac2
0,13231,0.001404,0.003247,0.000683,0.000609,0.006776,0.007471,0.874558,0.124164,0.080439,...,0.002448,0.002529,0.001228,0.001078,0.127339,0.135465,0.851654,0.837496,0.148346,0.162504
1,49051,0.007419,0.004546,0.001256,0.000986,0.005608,0.008095,0.508298,0.246679,0.053108,...,0.002547,0.002994,0.003168,0.002258,0.225505,0.336094,0.781144,0.663686,0.218856,0.336314
2,53021,0.002807,0.007738,0.003203,0.003786,0.012516,0.015469,0.560135,0.389494,0.086217,...,0.005912,0.007098,0.004517,0.006305,0.266644,0.207564,0.619399,0.627520,0.380601,0.372480
3,31119,0.004211,0.005238,0.001801,0.001570,0.010026,0.011305,0.781600,0.177663,0.079724,...,0.004209,0.004345,0.003759,0.004288,0.295720,0.242239,0.754142,0.717214,0.245858,0.282786
4,12045,0.001604,0.002721,0.000933,0.000906,0.005137,0.005581,0.756603,0.227601,0.034108,...,0.002148,0.002214,0.001152,0.001256,0.169199,0.144709,0.720895,0.715817,0.279105,0.284183


In [7]:
# Read in labels
labels = pd.read_csv("../label_process/labels.csv")
normalized_df["Labels"] = labels["Label"]

In [8]:
table = normalized_df[normalized_df['Labels'].notnull()]
feats = list(table.columns)
feats.remove("Fips")
feat_table = table[feats]
all_features = np.array(feat_table[feats])
X = all_features[:,:-1]
y = all_features[:,-1]


In [17]:
logistic_model = LogisticRegression(solver='lbfgs', max_iter=10000)
model = SelectFromModel(logistic_model).fit(X,y)
X_new = model.transform(X)
important_feats = model.get_support()
important_feats = np.append(important_feats, [False]) # To not include labels
print(feat_table.columns[important_feats])

new_logistic = LogisticRegression(solver='lbfgs', max_iter=10000).fit(X_new, y)
new_logistic.score(X_new,y)

Index(['Precincts', 'Republicans 08 (Votes)', 'Republicans 12 (Votes)',
       'Less Than High School Diploma', 'School Enrollment',
       'Native American Population', 'Total Population',
       'Preschool.Enrollment.Ratio.enrolled.ages.3.and.4', 'Gini.Coefficient',
       'Management.professional.and.related.occupations',
       'Service.occupations',
       'Construction.extraction.maintenance.and.repair.occupations',
       'Amerindian', 'White  Asian', 'Sire Homogeneity', 'lon', 'lat',
       'Poor.physical.health.days', 'Teen.births', 'Adult.smoking',
       'Sexually.transmitted.infections', 'Uninsured', 'Unemployment',
       'Violent.crime', 'Injury.deaths', 'Infant.mortality', 'CA', 'MAR',
       'ACFS', 'Max Alc', 'Summer Prcp', 'Autumn Prcp', 'Annual Tmax',
       'Winter Tmin', 'Summer Tavg', 'Summer Tmax', 'Spring Tavg',
       'Spring Tmin', 'Autumn Tmax', 'reporting', 'Votes16 Trumpd',
       'Votes16 Johnsong', 'Votes16 Castled', 'Est Votes Remaining',
       'Votes16

0.5568141592920354